In [ ]:
#hide
#skip
%config Completer.use_jedi = False
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [ ]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab(): 
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON

# Async Testing
> Testing async environment execution in a separate notebook.

One of the most important things to have working with the datablock is to answer: 

    Can we run multiple envs, in multiple workers, using a torch Module, all loaded on cuda?
    
You can however there are some important things to use in your notebook.

> Important: This is only important if you are training with `num_workers>0` and `str(default_device())=='cuda'`. If one of these is not true, then feel free
to train your model the *normal* way.

Relevant links:

  [setting the `MKL_THREADING_LAYER` variable](https://github.com/pytorch/pytorch/issues/37377)
  
  [handling `set_start_method('spawn')`](https://stackoverflow.com/questions/48822463/how-to-use-pytorch-multiprocessing)
  
  [using `if __name__ == '__main__'`](https://github.com/pytorch/pytorch/issues/3492)

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
from fastai.data.load import DataLoader,_FakeLoader,multiprocessing,_MultiProcessingDataLoaderIter

In [ ]:
from torch.utils.data._utils import worker as z

In [ ]:
from fastai.learner import Learner

In [ ]:
from fastai.test_utils import synth_learner
from fastrl.data.block import *

In [ ]:
if __name__ == '__main__':
    import os
    os.environ['MKL_THREADING_LAYER']="GNU"
    
    import torch.multiprocessing as mp
    try:mp.set_start_method('spawn',force=True)
    except Exception:pass
    
    # Third party libs
    from fastai.torch_basics import *
    from fastai.data.all import *
    # Local modules
    from fastrl.data.block import *
    from torch.optim import Adam

    
    dqn=DQN().share_memory()
    opt=Adam(dqn.parameters(),lr=0.01)
    source=ExperienceSource('CartPole-v1',None,steps_count=3,n_envs=1)
    ds=SourceDataset(source)
    
    dl=DataLoader(ds,num_workers=2,batch_size=5,n=100,persistent_workers=True)
    for _ in range(4):
        for x in dl:
            out=dqn.policy(x[0]['state'])+1
            target=torch.ones(out.shape)+1
            loss=nn.MSELoss()(out,target)
            opt.zero_grad()
            loss.backward()
            opt.step()

In [ ]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab(): 
    from nbdev.export import *
    from nbdev.export2html import *
    from nbdev.cli import make_readme
    make_readme()
    notebook2script()
    notebook2html()

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
Converted 00_core.ipynb.
Converted 00_nbdev_extension.ipynb.
Converted 05_data.block.ipynb.
Converted 05_data.test_async.ipynb.
Converted 20_test_utils.ipynb.
Converted index.ipynb.
Converted nbdev_template.ipynb.
